# Recommendation system to obtain the next best movement in 2048 Game

2048 is often played on a gray 4×4 grid, with numbered tiles that slide when a player moves them using the four arrow keys. Every turn, a new tile will randomly appear in an empty spot on the board with a value of 2. Tiles slide as far as possible in the chosen direction until they are stopped by either another tile or the edge of the grid. If two tiles of the same number collide while moving, they will merge into a tile with the total value of the two tiles that collided. The resulting tile cannot merge with another tile again in the same move. The intend of this project is to build a neural network using neuroevolution to add a component that improves the game experience, in our case, we implement a recommendation system to obtain the next best movement that can be done in the game to make an easier user experience.

Authors: Carlos Domínguez Becerril and Aitor González Marfil

### Imports

In [1]:
import pickle

# Libraries to work with numbers
import numpy as np
from scipy.special import softmax
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import math
import random

plt.rcParams['figure.figsize'] = [5, 5]
plt.rcParams['figure.dpi'] = 100 

In [2]:
!pip install neat
!pip install neat-python

     |████████████████████████████████| 88 kB 1.0 MB/s 
     |████████████████████████████████| 114 kB 4.4 MB/s 
  Created wheel for neat: filename=neat-0.4.1-py3-none-any.whl size=8078 sha256=ed920b774537515619dc0247697264b249a8819cdcd93d226278f0eadf39bd83
  Stored in directory: /root/.cache/pip/wheels/12/e6/c7/c5ea0eb2ce1c5f3f4385e9c21b2b2826388aafd029b888d612
Successfully built neat
     |████████████████████████████████| 44 kB 324 kB/s 


### 2048 game

The implementation of the game has been taken from https://github.com/yangshun/2048-python developed by Yangshun Tay.

The code of the game consists in 3 files:

- Constants.py: Includes the constants that the GUI is going to use. It includes: colors, valid keys, fonts, grid size, window size ...

- Logic.py: Includes the logic of the game that allows to perform the different movements, tiles merging, creation of new tiles ...

- Puzzle.py: Includes the GUI of the game.

In [3]:
SIZE = 400
GRID_LEN = 4
GRID_PADDING = 10

BACKGROUND_COLOR_GAME = "#92877d"
BACKGROUND_COLOR_CELL_EMPTY = "#9e948a"

BACKGROUND_COLOR_DICT = {2: "#eee4da", 4: "#ede0c8", 8: "#f2b179",
                         16: "#f59563", 32: "#f67c5f", 64: "#f65e3b",
                         128: "#edcf72", 256: "#edcc61", 512: "#edc850",
                         1024: "#edc53f", 2048: "#edc22e",

                         4096: "#eee4da", 8192: "#edc22e", 16384: "#f2b179",
                         32768: "#f59563", 65536: "#f67c5f", }

CELL_COLOR_DICT = {2: "#776e65", 4: "#776e65", 8: "#f9f6f2", 16: "#f9f6f2",
                   32: "#f9f6f2", 64: "#f9f6f2", 128: "#f9f6f2",
                   256: "#f9f6f2", 512: "#f9f6f2", 1024: "#f9f6f2",
                   2048: "#f9f6f2",

                   4096: "#776e65", 8192: "#f9f6f2", 16384: "#776e65",
                   32768: "#776e65", 65536: "#f9f6f2", }

FONT = ("Verdana", 40, "bold")

KEY_UP_ALT = "\'\\uf700\'"
KEY_DOWN_ALT = "\'\\uf701\'"
KEY_LEFT_ALT = "\'\\uf702\'"
KEY_RIGHT_ALT = "\'\\uf703\'"

KEY_UP = "'w'"
KEY_DOWN = "'s'"
KEY_LEFT = "'a'"
KEY_RIGHT = "'d'"
KEY_BACK = "'b'"

KEY_J = "'j'"
KEY_K = "'k'"
KEY_L = "'l'"
KEY_H = "'h'"

In [4]:
#
# CS1010FC --- Programming Methodology
#
# Mission N Solutions
#
# Note that written answers are commented out to allow us to run your
# code easily while grading your problem set.

#######
#Task 1a#
#######

# [Marking Scheme]
# Points to note:
# Matrix elements must be equal but not identical
# 1 mark for creating the correct matrix


def new_game(n):
    matrix = []

    for i in range(n):
        matrix.append([0] * n)
    return matrix

###########
# Task 1b #
###########

# [Marking Scheme]
# Points to note:
# Must ensure that it is created on a zero entry
# 1 mark for creating the correct loop


def add_two(mat):
    a = random.randint(0, len(mat)-1)
    b = random.randint(0, len(mat)-1)
    while(mat[a][b] != 0):
        a = random.randint(0, len(mat)-1)
        b = random.randint(0, len(mat)-1)
    mat[a][b] = 2
    return mat

###########
# Task 1c #
###########

# [Marking Scheme]
# Points to note:
# Matrix elements must be equal but not identical
# 0 marks for completely wrong solutions
# 1 mark for getting only one condition correct
# 2 marks for getting two of the three conditions
# 3 marks for correct checking


def game_state(mat):
    for i in range(len(mat)):
        for j in range(len(mat[0])):
            if mat[i][j] == 2048:
                return 'win'
    for i in range(len(mat)-1):
        # intentionally reduced to check the row on the right and below
        # more elegant to use exceptions but most likely this will be their solution
        for j in range(len(mat[0])-1):
            if mat[i][j] == mat[i+1][j] or mat[i][j+1] == mat[i][j]:
                return 'not over'
    for i in range(len(mat)):  # check for any zero entries
        for j in range(len(mat[0])):
            if mat[i][j] == 0:
                return 'not over'
    for k in range(len(mat)-1):  # to check the left/right entries on the last row
        if mat[len(mat)-1][k] == mat[len(mat)-1][k+1]:
            return 'not over'
    for j in range(len(mat)-1):  # check up/down entries on last column
        if mat[j][len(mat)-1] == mat[j+1][len(mat)-1]:
            return 'not over'
    return 'lose'

###########
# Task 2a #
###########

# [Marking Scheme]
# Points to note:
# 0 marks for completely incorrect solutions
# 1 mark for solutions that show general understanding
# 2 marks for correct solutions that work for all sizes of matrices


def reverse(mat):
    new = []
    for i in range(len(mat)):
        new.append([])
        for j in range(len(mat[0])):
            new[i].append(mat[i][len(mat[0])-j-1])
    return new

###########
# Task 2b #
###########

# [Marking Scheme]
# Points to note:
# 0 marks for completely incorrect solutions
# 1 mark for solutions that show general understanding
# 2 marks for correct solutions that work for all sizes of matrices


def transpose(mat):
    new = []
    for i in range(len(mat[0])):
        new.append([])
        for j in range(len(mat)):
            new[i].append(mat[j][i])
    return new

##########
# Task 3 #
##########

# [Marking Scheme]
# Points to note:
# The way to do movement is compress -> merge -> compress again
# Basically if they can solve one side, and use transpose and reverse correctly they should
# be able to solve the entire thing just by flipping the matrix around
# No idea how to grade this one at the moment. I have it pegged to 8 (which gives you like,
# 2 per up/down/left/right?) But if you get one correct likely to get all correct so...
# Check the down one. Reverse/transpose if ordered wrongly will give you wrong result.


def cover_up(mat):
    new = []
    for j in range(GRID_LEN):
        partial_new = []
        for i in range(GRID_LEN):
            partial_new.append(0)
        new.append(partial_new)
    done = False
    for i in range(GRID_LEN):
        count = 0
        for j in range(GRID_LEN):
            if mat[i][j] != 0:
                new[i][count] = mat[i][j]
                if j != count:
                    done = True
                count += 1
    return (new, done)


def merge(mat):
    done = False
    for i in range(GRID_LEN):
        for j in range(GRID_LEN-1):
            if mat[i][j] == mat[i][j+1] and mat[i][j] != 0:
                mat[i][j] *= 2
                mat[i][j+1] = 0
                done = True
    return (mat, done)


def up(game):
    # print("up")
    # return matrix after shifting up
    game = transpose(game)
    game, done = cover_up(game)
    temp = merge(game)
    game = temp[0]
    done = done or temp[1]
    game = cover_up(game)[0]
    game = transpose(game)
    return (game, done)


def down(game):
    # print("down")
    game = reverse(transpose(game))
    game, done = cover_up(game)
    temp = merge(game)
    game = temp[0]
    done = done or temp[1]
    game = cover_up(game)[0]
    game = transpose(reverse(game))
    return (game, done)


def left(game):
    # print("left")
    # return matrix after shifting left
    game, done = cover_up(game)
    temp = merge(game)
    game = temp[0]
    done = done or temp[1]
    game = cover_up(game)[0]
    return (game, done)


def right(game):
    # print("right")
    # return matrix after shifting right
    game = reverse(game)
    game, done = cover_up(game)
    temp = merge(game)
    game = temp[0]
    done = done or temp[1]
    game = cover_up(game)[0]
    game = reverse(game)
    return (game, done)

#### Puzzle V2 

This is a modified version of puzzle.py from https://github.com/yangshun/2048-python/blob/master/puzzle.py that removes the GUI part leaving only the core of the game and some extra functions to train the neural network.

New Functions:
- print_board: This function prints the board of the game in the standard output.
- flatten: This function flattens the board of the game in a 1 dimensional array.
- get_state: This function returns the state of the game "not over", "win" or "lose"
- valid_movements: This function return the valid movements of the game in a vector where 0 is not valid and 1 is valid.
- fitness: fitness function to train the neural network according to the smoothness, monotonicity, number of empty cells, maximum value in the matrix and sum of the values in the matrix. Taken from Ovolve user. https://github.com/ovolve/2048-AI

In [5]:
class game:
    # Game matrix initialization and available commands.
    def __init__(self):
        self.commands = {0: up, 1: left, 2: down, 3: right}
        self.init_matrix()
    
    # Perform an action in the game
    def key_down(self, key, std_out=False):
        if key in self.commands:
            # make the movement
            self.matrix, done = self.commands[key](self.matrix)
            if done:
                self.matrix = add_two(self.matrix)
                done = False
                board_state = self.get_state()
                if board_state == 'win':
                    print("YOU WIN!")
                if board_state == 'lose':
                    if std_out:
                        print("YOU LOSE!")
    
    # Initialization of the game matrix
    def init_matrix(self):
        self.matrix = new_game(GRID_LEN)
        self.matrix = add_two(self.matrix)
        self.matrix = add_two(self.matrix)  
    
    # Print the board in the standard output
    def print_board(self):
        for i in self.matrix:
            print(" ".join([str(int) for int in i]))
    
    # Convert the matrix in a 1 dimensional array
    def flatten(self):
        return np.array(self.matrix).flatten()
    
    # Get the state of the game ("not over", "win", "lose")
    def get_state(self):
        return game_state(self.matrix)
    
    # Return the valid movements that we can perform
    def valid_movements(self):
        valid = []
        for i in [0, 1, 2, 3]:
            
            # Save the old matrix so we can return to the previous state without altering the game
            old = list(self.matrix)
        
            # Apply the movement
            self.key_down(i, std_out=False)
            
            # Obtain the new matrix
            new = list(self.matrix)
            
            # Check whether the old and new matrix are the same. 
            # Same -> 0 (movement not valid)
            # Different -> 1 (valid movement)
            if old != new:
                valid.append(1)
            else:
                valid.append(0)
            self.matrix = old
        return valid
    
    # This function returns the fitness of the matrix according to the smoothness, 
    # monotonicity, number of empty cells, maximum value in the matrix and sum of the values in the matrix.
    # Taken from Ovolve user. https://github.com/ovolve/2048-AI
    def fitness(self):
        vectors = [
          (0,  -1), # up
          (1,  0 ), # right
          (0,  1 ), # down
          (-1, 0 ) # left
        ]
        
        # this function counts the number of zeros in the matrix
        def count_0(m):
            count = 0
            for i in m:
                for j in i:
                    if j == 0:
                        count += 1
            return count
        
        # measures how monotonic the grid is. This means the values of the tiles are strictly increasing
        # or decreasing in both the left/right and up/down directions
        def monotonicity2(m):
            # scores for all four directions
            totals = [0, 0, 0, 0]

            # up/down direction
            for x in range(4): 
                current = 0
                next_Dir = current+1
                while ( next_Dir<4 ) :
                    while ( next_Dir<4 and not m[x][next_Dir]!=0 ):
                        next_Dir+=1

                    if (next_Dir>=4):
                        next_Dir-=1
                    if m[x][current]!=0:
                        currentValue =  math.log(m[x][current]) / math.log(2) 
                    else:
                        currentValue = 0

                    if m[x][next_Dir]!=0:
                        nextValue = math.log(m[x][next_Dir]) / math.log(2)
                    else:
                        nextValue = 0

                    if (currentValue > nextValue):
                        totals[0] += nextValue - currentValue
                    elif (nextValue > currentValue):
                        totals[1] += currentValue - nextValue

                    current = next_Dir
                    next_Dir+=1


          # left/right direction
            for y in range(4):
                current = 0
                next_Dir = current+1
                while ( next_Dir<4 ):
                    while  next_Dir<4 and not m[next_Dir][y]!=0:
                        next_Dir+=1

                    if (next_Dir>=4):
                        next_Dir-=1

                    if m[current][y]!=0:
                        currentValue =  math.log(m[current][y]) / math.log(2)
                    else:
                        currentValue = 0

                    if m[next_Dir][y]!=0:
                        nextValue = math.log(m[next_Dir][y]) / math.log(2)
                    else:
                        nextValue = 0

                    if (currentValue > nextValue):
                        totals[2] += nextValue - currentValue
                    elif (nextValue > currentValue):
                        totals[3] += currentValue - nextValue

                    current = next_Dir;
                    next_Dir+=1

            return max(totals[0], totals[1]) + max(totals[2], totals[3])
        
        # Progress towards the vector direction until an obstacle is found
        def findFarthestPosition(m, x, y, dir_x, dir_y):
            while (x>=0 and x<4) and (y>=0 and y<4):
                x += dir_x
                y += dir_y
            return x - dir_x, y - dir_y


        # measures how smooth the grid is (as if the values of the pieces
        # were interpreted as elevations). Sums of the pairwise difference
        # between neighboring tiles (in log space, so it represents the
        # number of merges that need to happen before they can merge). 
        # Note that the pieces can be distant
        def smoothness(m):
            smoothness = 0
            for x in range(4):
                for y in range(4):
                    if ( m[x][y]!= 0):
                        value = math.log(m[x][y]) / math.log(2)
                        direction=0
                        while (direction<=3):
                            vec_x, vec_y = vectors[direction]
                            targetCell_x, targetCell_y = findFarthestPosition(m, x, y, vec_x, vec_y)
                            if ( m[targetCell_x][targetCell_y] != 0 ):
                                target_value = m[targetCell_x][targetCell_y]
                                targetValue = math.log(target_value) / math.log(2)
                                smoothness -= abs(value - targetValue)
                            direction+=1
            return smoothness
        emptyCells = count_0(self.matrix)

        smoothWeight = 0.1
        mono2Weight  = 1.0
        emptyWeight  = 2.7
        maxWeight    = 1.0
        sumWeight    = 0.03

        a = smoothness(self.matrix) * smoothWeight
        b = monotonicity2(self.matrix) * mono2Weight
        if emptyCells == 0:
            c = -10 * emptyWeight
        else:
            c = math.log(emptyCells) * emptyWeight
        d = max(map(max, self.matrix)) * maxWeight
        e = sum([sum(i) for i in self.matrix]) * sumWeight

        return a + b + c + d + e

### Training the neural network

First we declare some global variables to be used through the whole notebook

In [6]:
# Name of the neat config file
config_file = "../input/mlnn-neat-files/neat_16.config"
# Name of the winner genome
winner_name = "./winner_genome.neat"
# Number of generations to train the neural network
generations = 1000
# Number of games to validate the neural network
validation = 10000
# Fitness function to use
fitness_function = 1 # Only 1 or 2
# whether to create checkpoints or not
checkpoints = False

Function that evaluates a generation

In [7]:
global biggest
biggest = 0

In [8]:
# Function that evaluates one generation
def eval_genomes(genomes, config):
    global biggest
    # List with the neural network of each genome
    nets = []
    # List with each Genome
    ge = []
    # List with the Game board of each genome
    boards = []
    
    # Load the neural networks, genomes and boards in the previous lists so we can access them with indexes
    for _, genome in genomes:
        # Create the neural network with the genome and save it in the net list
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)
        # Create a new game
        boards.append([game(), False])
        # Set the genome fitness to 0 and save it in the ge list
        genome.fitness = 0
        ge.append(genome)
    
    run, all_over = True, len(boards)
    # Execute until no more movements can be perform in any of the boards
    while run:
        for x, (board, over) in enumerate(boards):
            # If the game is not over
            if not over:
                # Copy the old board
                matrix = board.flatten()
                
                # check the valid movements
                valid = np.array(board.valid_movements())
                
                # Use the neural network and get the output layer
                # The input is the matrix of the game normalized
                output = nets[x].activate(matrix)

                # Apply softmax layer to the last output (a layer with 4 outputs)
                output = softmax(output)
                
                # Multiply the output by the valid movements so we obtain only valid movements.
                output = np.multiply(output, valid)
                
                # Use the index with the highest number as the key
                key = np.argmax(output)

                # Apply one action
                board.key_down(key)
                
                # Get the state
                board_state = board.get_state()
                
                # Update the fitness according to the game state
                if board_state == "lose":
                    boards[x][1] = True
                    all_over -= 1
                    
                    if fitness_function == 1:
                        ge[x].fitness = ge[x].fitness / 2
                    else:
                        ge[x].fitness = board.fitness() - 400
                        
                elif board_state == "win":
                    boards[x][1] = True
                    all_over -= 1
                    if fitness_function == 1:
                        ge[x].fitness = (5000 - ge[x].fitness) * 2
                    else:
                        ge[x].fitness = board.fitness() + 500
                else:
                    if fitness_function == 1:
                        ge[x].fitness += 1
                    else:
                        ge[x].fitness = board.fitness()
                
        # If all the games are over we finish this generation   
        if all_over == 0:
            run = False
    gen = None
    k = 0
    for x, genome in genomes:
        if genome.fitness > biggest:
            gen = nets[k]
            biggest = genome.fitness
        k += 1
    if gen != None:
        # Save the genome with the best fitness
        with open("./genome"+str(int(biggest))+".neat", "wb") as f:
            pickle.dump(gen, f)

### Neat configuration

In [9]:
import neat

In [ ]:
configuration = neat.config.Config(neat.DefaultGenome, 
                                   neat.DefaultReproduction,
                                   neat.DefaultSpeciesSet,  
                                   neat.DefaultStagnation,
                                   config_file)
# Create the population
population = neat.Population(configuration)

# Print stats of the population
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)
# Add reporter to obtain checkpoints
if checkpoints:
    population.add_reporter(neat.Checkpointer(1))
population.run(eval_genomes, generations)

Save the winner

In [ ]:
# Save the genome with the best fitness
with open(winner_name, "wb") as f:
    pickle.dump(winner, f)

### Validation

In [ ]:
config = neat.config.Config(neat.DefaultGenome, 
                            neat.DefaultReproduction, 
                            neat.DefaultSpeciesSet, 
                            neat.DefaultStagnation, 
                            config_file)
# Unpickle saved winner
with open(winner_name, "rb") as f:
    genome = pickle.load(f)

# Convert loaded genome into required data structure
genomes = [(1, genome)]
        
# Feed Fordward Network
net = neat.nn.FeedForwardNetwork.create(genome, config)

In [ ]:
max_values = {}
steps = []
for i in range(validation):
    # print("STEP: ", i, " of ", validation)
    g = game()
    movements = 0
    while g.get_state() == "not over":
        
        # Obtain the matrix    
        matrix = g.flatten()
        # Obtain the valid movements
        valid = np.array(g.valid_movements())
        # Use the neural network and get the output layer
        output = net.activate(matrix)
        # output = net.activate(np.concatenate((matrix, valid)))
        # Apply softmax layer to the last output
        output = softmax(output)
        # Use the index with the highest number as the key
        output = np.multiply(output, valid)                                      
        # Use the output with highest number as our key
        key = np.argmax(output, axis=0)
        # Apply the movement
        g.key_down(key)
        
        movements += 1
        
    maximum = max(map(max, g.matrix))
    steps.append(movements)
    if maximum in max_values:
        max_values[maximum] += 1
    else:
        max_values[maximum] = 1                

In [ ]:
print("max_value / number_of_times", sorted(list(max_values.items())))
print("AVG number of movements", sum(steps)/ len(steps))

In [ ]:
def func(pct, allvals):
    absolute = int(pct/100.*np.sum(allvals))
    return "{:.1f}%\n({:d} times)".format(pct, absolute)

# Pie chart, where the slices will be ordered and plotted counter-clockwise:

labels = [str(i[0]) for i in max_values.items()]
sizes = [i[1] for i in max_values.items()]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct=lambda pct: func(pct, sizes),
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.xlabel("Maximum puntuation obtained in " + str(validation) + " games")
plt.legend()
plt.show()

In [ ]:
# Bar plot
plt.bar(labels, sizes)
plt.xlabel("Maximum puntuation obtained in " + str(validation) + " games")
plt.show()